<a href="https://colab.research.google.com/github/VOX304/SchoolChatbot/blob/main/Preprocessing/Colab_preprocessing/Image_Prep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install paddleocr ultralytics opencv-python numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
  

In [2]:
!pip install pdf2image python-docx pdfplumber paddlepaddle-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 759.0/759.0 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 66.3 MB/s eta 0:00:00
  Attempting uninstall: opt-einsum
    Found existing installation: opt_einsum 3.4.0
    Uninstalling opt_einsum-3.4.0:
      Successfully uninstalled opt_einsum-3.4.0


In [3]:
!pip install paddlepaddle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 MB 5.3 MB/s eta 0:00:00


In [4]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 45.5 MB/s eta 0:00:00


In [8]:
!pip install ultralytics


In [9]:
import os
import fitz  # PyMuPDF
import json
from docx import Document
from ultralytics import YOLO


model = YOLO("yolov8s.pt")  # Use the small YOLOv8 model (pretrained)


def is_image_page(page):
    # Convert the PDF page to an image
    img = page.get_pixmap()
    img = Image.frombytes("RGB", [img.width, img.height], img.samples)

    # Run YOLOv8 inference
    results = model(img)

    # Check if any objects are detected
    return len(results[0].boxes) > 0  # ✅ Corrected access to bounding boxes


def extract_images_from_pdf(pdf_path, output_folder="images1"):
    """Extract images from a PDF file using PyMuPDF, skipping text pages."""
    os.makedirs(output_folder, exist_ok=True)
    doc = fitz.open(pdf_path)  # Open the PDF

    for page_num in range(len(doc)):
        page = doc[page_num]

        # Step 1: Check if page is image-based using YOLO
        if not is_image_page(page):
            continue  # Skip non-image pages

        # Step 2: Extract images from image-based pages
        images = page.get_images(full=True)
        if not images:
            continue  # Skip if no images found

        for img_index, img in enumerate(images):
            xref = img[0]  # Image XREF
            img_data = doc.extract_image(xref)  # Extract image data
            img_bytes = img_data.get("image", None)  # Get raw image bytes

            if img_bytes:
                img_ext = img_data["ext"]  # Image format
                img_path = os.path.join(output_folder, f"pdf_img_{page_num + 1}_{img_index}.{img_ext}")
                with open(img_path, "wb") as f:
                    f.write(img_bytes)

    doc.close()

def extract_images_from_docx(docx_path, output_folder="images1"):
    """Extract images from a DOCX file and store metadata."""
    os.makedirs(output_folder, exist_ok=True)
    image_metadata = []
    doc = Document(docx_path)

    for i, rel in enumerate(doc.part.rels.values()):
        if "image" in rel.reltype:
            img_data = rel.target_part.blob
            img_path = os.path.join(output_folder, f"docx_img_{i}.jpg")
            with open(img_path, "wb") as f:
                f.write(img_data)

            # Store metadata for VectorDB
            image_metadata.append({
                "document_name": os.path.basename(docx_path),
                "document_type": "docx",
                "Data_type": "image",
                "image_index": i,
                "image_path": img_path
            })

    return image_metadata

def process_documents_in_folder(document_folder, output_json="image_metadata.json"):
    """Process all PDF and DOCX files in a folder and save metadata."""
    all_metadata = []
    os.makedirs(document_folder, exist_ok=True)


    for file_name in os.listdir(document_folder):
        file_path = os.path.join(document_folder, file_name)
        if file_name.endswith(".pdf"):
            image_metadata = extract_images_from_pdf(file_path)
        elif file_name.endswith(".docx"):
            image_metadata = extract_images_from_docx(file_path)
        else:
            continue  # Skip unsupported file types

        all_metadata.extend(image_metadata)

    '''
    # Save metadata to JSON for easy VectorDB insertion
    with open(output_json, "w", encoding="utf-8") as f:
        json.dump(all_metadata, f, ensure_ascii=False, indent=4)

    return all_metadata'''

# Example usage
document_folder = "/content/sample_data/TCU"  # Replace with your folder path
metadata_results = process_documents_in_folder(document_folder)
print("✅ Metadata saved to image_metadata.json")


100%|██████████| 21.5M/21.5M [00:00<00:00, 81.7MB/s]



image 1/1 /content/temp_page.png: 640x480 1 book, 39.6ms
Speed: 20.6ms preprocess, 39.6ms inference, 351.2ms postprocess per image at shape (1, 3, 640, 480)


AttributeError: 'list' object has no attribute 'xyxy'

In [ ]:

import os

def check_files_in_folder(folder_path):
    """Kiểm tra thư mục có chứa file không"""
    if not os.path.exists(folder_path):
        print(f"⚠️ Thư mục '{folder_path}' không tồn tại!")
        return False

    files = os.listdir(folder_path)

    if not files:
        print(f"📂 Thư mục '{folder_path}' rỗng!")
        return False
    else:
        print(f"✅ Thư mục '{folder_path}' có {len(files)} tệp tin:")
        for file in files:
            print(f"   📄 {file}")
        return True

# 📌 Đường dẫn thư mục cần kiểm tra
folder_path = "/content/sample_data/tcu"
check_files_in_folder(folder_path)
